In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import random_projection
import lightgbm as lgb
from math import exp, log
path = '/home/ec2-user/Sanjay/main/santander/'

/home/ec2-user/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
X_train = pd.read_csv(path + 'codes/data/' + '1_raw_data_train.csv')
X_test = pd.read_csv(path + 'codes/data/' + '1_raw_data_test.csv')

test_df = pd.read_csv(path + 'test.csv',  usecols=['ID'])
train_df = pd.read_csv(path + 'train.csv', usecols=['target'])
y_train = train_df[['target']]['target'].map(lambda x : log(1+x))
X_train.shape, X_test.shape, train_df.shape

In [6]:
Random_Projection_train = pd.read_csv(path + 'codes/data/' + '2_Random_Projection_train.csv')
Random_Projection_test = pd.read_csv(path + 'codes/data/' + '2_Random_Projection_test.csv')

kmeans_train = pd.read_csv(path + 'codes/data/' + '3_kmeans_train.csv')
kmeans_test = pd.read_csv(path + 'codes/data/' + '3_kmeans_test.csv')

aggr_train = pd.read_csv(path + 'codes/data/' + '4_aggr_train.csv')
aggr_test = pd.read_csv(path + 'codes/data/' + '4_aggr_test.csv')

pca_train = pd.read_csv(path + 'codes/data/' + '5_pca_train.csv')
pca_test = pd.read_csv(path + 'codes/data/' + '5_pca_test.csv')

print(Random_Projection_train.shape, Random_Projection_test.shape, 
      kmeans_train.shape, kmeans_test.shape,
      aggr_train.shape, aggr_test.shape, 
      pca_train.shape, pca_test.shape )

(4459, 50) (49343, 50) (4459, 9) (49343, 9) (4459, 9) (49343, 9) (4459, 20) (49343, 20)


In [7]:
train = X_train.join(Random_Projection_train).join(kmeans_train).join(aggr_train).join(pca_train)
test = X_test.join(Random_Projection_test).join(kmeans_test).join(aggr_test).join(pca_test)
train.shape, test.shape

((4459, 4818), (49343, 4818))

#### 10 fold CV LightGBM

In [11]:
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 100,
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_seed' : 42,
    'bagging_freq': 5,
    'verbose': 50
}

In [12]:
def lgbm_model(X_tr, y_tr, X_va, y_va, test, fold):
    tr_data = lgb.Dataset(X_tr, label=y_tr)
    va_data = lgb.Dataset(X_va, label=y_va)
    
    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=4000,
                      early_stopping_rounds=120,
                      verbose_eval=300)
    
    y_pred = model.predict(X_va)
    valid = pd.DataFrame(y_pred, columns=['prob' + str(fold)])
    test_pred = model.predict(test)
    test = pd.DataFrame(test_pred, columns=['prob' + str(fold)])
    return test

In [15]:
from sklearn.model_selection import KFold
X = train
y = y_train
kf = KFold(n_splits=10, shuffle=True, random_state=2017)
kf.get_n_splits(X), kf.split(X)
fold=1
valid_final = test_df[['ID']]
for train_idx, test_idx in kf.split(X):
    print("FOLD ", fold, " ----------------------------------------------------------------------------------")
    X_tr = X[X.index.isin(train_idx)]
    y_tr = y[y.index.isin(train_idx)]
    X_va = X[X.index.isin(test_idx)]
    y_va = y[y.index.isin(test_idx)]
    print(X_tr.shape, y_tr.shape, X_va.shape, y_va.shape)
    valid = lgbm_model(X_tr, y_tr, X_va, y_va, test, fold)
    valid_final = valid_final.join(valid)
    print(valid.shape, valid_final.shape)
    fold = fold+1

FOLD  1  ----------------------------------------------------------------------------------
(4013, 4818) (4013,) (446, 4818) (446,)
Training until validation scores don't improve for 120 rounds.
[300]	valid_0's rmse: 1.35284
Early stopping, best iteration is:
[365]	valid_0's rmse: 1.34921
(49343, 1) (49343, 2)
FOLD  2  ----------------------------------------------------------------------------------
(4013, 4818) (4013,) (446, 4818) (446,)
Training until validation scores don't improve for 120 rounds.
[300]	valid_0's rmse: 1.36369
[600]	valid_0's rmse: 1.33836
Early stopping, best iteration is:
[645]	valid_0's rmse: 1.33776
(49343, 1) (49343, 3)
FOLD  3  ----------------------------------------------------------------------------------
(4013, 4818) (4013,) (446, 4818) (446,)
Training until validation scores don't improve for 120 rounds.
[300]	valid_0's rmse: 1.34484
[600]	valid_0's rmse: 1.29242
Early stopping, best iteration is:
[715]	valid_0's rmse: 1.28887
(49343, 1) (49343, 4)
FOLD

In [16]:
valid_final['target'] = valid_final.iloc[:,1:fold].mean(axis=1)
valid_final['target'] = valid_final['target'].map(lambda x : exp(x)-1)

In [17]:
valid_final = valid_final.iloc[:49342]
valid_final.to_csv(path + 'submit/' + 'lgbm_10fold_1.csv', index=False)
valid_final[['ID', 'target']].to_csv(path + 'submit/' + 'lgbm_1.csv', index=False)

### Predict the nearby input

In [7]:
pred = pd.read_csv(path + 'submit/' + 'lgbm_1.csv')
pred_input = pd.read_csv(path + 'test.csv')
pred = pred.merge(pred_input, on='ID', how='inner')
pred.shape, pred_input.shape

In [16]:
def nearby_value(array, value):
    idx,val = min(enumerate(np.sort(array[array!=0])), key=lambda x: abs(x[1]-value))
    return val

In [18]:
close_value = []
for i in range(pred.shape[0]):
    if (i%5000)==0:
        print(i)
    close_value.append(nearby_value(pred.iloc[0][pred.columns[1:]].values, pred.target[i]))

In [22]:
pred['target'] = close_value

In [24]:
pred[['ID', 'target']].to_csv(path + 'submit/' + 'lgbm_1_nearby_input.csv', index=False)

### Ensemble

In [9]:
import pandas as pd
path = '/home/ec2-user/Sanjay/main/santander/'
data1 = pd.read_csv(path + 'submit/' + 'baseline_sub_lag_37.csv')
data2 = pd.read_csv(path + 'submit/' + 'check_blend1.csv')

In [10]:
import copy
data = data1.copy()
data['target'] = data1['target']*0.7 + data2['target']*0.3

In [11]:
data[['ID', 'target']].to_csv(path + 'submit/' + 'check_blend1.csv', index=False)